In [61]:
%matplotlib notebook
import scipy
import sympy
from modsim import *
import numpy as np
# Average statistics.
# Grams of ethanol consumed.
alcohol_consumed = 15
# Absorption rate constant. 3.18 is mean blood value.
k = 3.18
# BAC. .04 is the legal driving limit, though you should never drink and drive!
BAC = .04 
# Elimination rate constant. .163 is mean blood value.
beta = .163
# Body fat ratio. .67 for women, .73 men.
ratio = .73
# Weight.
weight = 70
# Liters of blood in human body
blood_amount = 5

class simplistic_model():
    """
    One-compartment model for alcohol absorption and elimination.
    """
    def __init__(self):
        """
        Constructor.
        """
        pass
    @staticmethod   
    def absorb_alcohol(d, t, w=weight):
        """
        Returns the amount of alcohol absorbed.
        """
        return ((d * (1-np.exp(-k*t)))/(ratio*w))
    
    @staticmethod   
    def eliminate_alcohol(t):
        """
        Returns the amount of alcohol eliminated
        """
        return beta * t

    @staticmethod   
    def model_alcohol_processing(d, t):
        """
        Simulates alcohol processing with a timeseries.
        TODO this is currently untested but pushing anyways.
        h = number of hours.
        d = dosage in grams of ethanol.
        """
        result = TimeSeries()
        for h in linrange(0, t):
            result[h] = (absorb_alcohol(d, (h)) - eliminate_alcohol((h)))/blood_amount
            if result[h] <= 0:
                result[h] = 0
        return result
    
    @staticmethod
    def model_processing_after_eating(d, t, time_of_eating, initial_weight, weight_gain):
        """
        
        """
        
        
    @staticmethod   
    def weight_based_on_bac(t, input_BAC):
        """
        Returns the weight of the individual based on static BAC and time.
        """
        return (alcohol_consumed*(1-np.exp(-k*t)))/((input_BAC + beta*t)*ratio)
    
    @staticmethod   
    def BAC_based_on_weight(t, input_weight):
        """
        Returns the BAC of the individual based on static weight and time.
        """
    
        return ((d*(1-np.exp(-k*t))/(ratio*input_weight)) - beta*t)

class two_compartment_model():
    """
    Two-compartment model for alcohol absorption and
    """
    def __init__():
        """
        Constructor.
        """
        pass
    @staticmethod
    
    def model_alcohol_processing(t, d):
        return gastric_emptying(d, t) - stomach_processing(d, t) - small_intestine_processing(d, t) - lean_body_mass_processing(d, t)
    
    @staticmethod
    def gastric_emptying(d, t):
        """
        Models alcohol processing in the stomach.
        """
        return ((-7.05)/((1 + (.42*(d/.470)**2)*t)))
   
    def stomach_processing_slope(c, t):
        print("stomach procssing slope")
        return -(5.55*c)/(1.00 + (0.42*c)**2)

    @staticmethod
    def stomach_processing(c, t):
        """
        Models alcohol processing in the stomach.
        """
        print("Processing")
        stomach_eq = scipy.integrate.odeint(two_compartment_model.stomach_processing_slope(c, t), c, t=linrange(1,40))

    @staticmethod
    def small_intestine_processing(d, t):
        pass
    
    @staticmethod
    def lean_body_mass_processing(d, t):
        pass
    


Processing
stomach procssing slope


error: The function and its Jacobian must be callable functions.